# Download e extração do dataset

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d a2015003713/militaryaircraftdetectiondataset

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('militaryaircraftdetectiondataset.zip', 'r')
zip_ref.extractall('/dataset')
zip_ref.close()

# Imports necessários

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, applications, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#Função para plotar histórico do modelo

In [ ]:
def plota_historico_modelo(historico_modelo):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    axs[0].plot(range(1,len(historico_modelo.history['accuracy'])+1),
                historico_modelo.history['accuracy'],'r')
    axs[0].plot(range(1,len(historico_modelo.history['val_accuracy'])+1),
                historico_modelo.history['val_accuracy'],'b')
    axs[0].set_title('Acurácia do Modelo')
    axs[0].set_ylabel('Acuracia')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1, len(historico_modelo.history['accuracy']) + 1, 1))

    axs[0].legend(['training accuracy', 'validation accuracy'], loc='best')

    axs[1].plot(range(1,len(historico_modelo.history['loss'])+1),
                historico_modelo.history['loss'],'r')
    axs[1].plot(range(1,len(historico_modelo.history['val_loss'])+1),
                historico_modelo.history['val_loss'],'b')
    axs[1].set_title('Perda/Loss do Modelo')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1, len(historico_modelo.history['loss']) + 1, 1))
    axs[1].legend(['training loss', 'validation Loss'], loc='best')
    fig.savefig('historico_modelo_mod01.png')
    plt.show()

# Carregando dados

In [ ]:
dataset_directory = '/dataset/crop'  # se rodar localmente, utilizar 'dataset/crop'

In [ ]:
batch_size = 40
image_size = (128, 128)

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_directory,
    validation_split=0.3,
    subset="training",
    shuffle=True,
    image_size=image_size,
    batch_size = batch_size,
    seed=123
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_directory,
    validation_split=0.3,
    subset="validation",
    shuffle=True,
    image_size=image_size,
    batch_size = batch_size,
    seed=123
)

Found 19270 files belonging to 43 classes.
Using 13489 files for training.
Found 19270 files belonging to 43 classes.
Using 5781 files for validation.


In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_data = validation_dataset.take(val_batches//5)
val_data = validation_dataset.skip(val_batches//5)

class_names = train_dataset.class_names

The prefetch method is used to improve data loading performance by asynchronously prefetching data samples from the input dataset.
The buffer_size parameter, set to AUTOTUNE, lets TensorFlow determine an appropriate buffer size for this prefetch operation based on
available system resources and other factors.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_prefetch = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_prefetch = val_data.prefetch(buffer_size=AUTOTUNE)
test_prefetch = test_data.prefetch(buffer_size=AUTOTUNE)

train_prefetch = train_prefetch.map(lambda x, y: (x, tf.one_hot(y, len(class_names))))
val_prefetch = val_data.map(lambda x, y: (x, tf.one_hot(y, len(class_names))))

# Treinando o modelo

In [ ]:
model = Sequential()

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(factor=0.4, fill_mode="wrap"),
    layers.experimental.preprocessing.RandomContrast(factor=0.2)
])

preprocess_input = applications.resnet50.preprocess_input

pretrained_model = ResNet50(
    include_top=False,
    input_shape=(128, 128, 3),
    weights='imagenet'
)
pretrained_model.trainable = False

global_avg = layers.GlobalAveragePooling2D()
output_layer = layers.Dense(len(class_names), activation='softmax')

In [ ]:
inputs = tf.keras.Input(shape=(128, 128, 3))
x = data_augmentation(inputs)
x = preprocess_input(inputs)
x = pretrained_model(x)
x = global_avg(x)
outputs = output_layer(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 tf.__operators__.getitem_4  (None, 128, 128, 3)       0         
  (SlicingOpLambda)                                              
                                                                 
 tf.nn.bias_add_4 (TFOpLamb  (None, 128, 128, 3)       0         
 da)                                                             
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d_4  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                           

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(optimizers.schedules.CosineDecay(0.0001, 500)),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_prefetch, validation_data=(val_prefetch), epochs = 5)

plota_historico_modelo(history)

In [ ]:
total_layers = len(pretrained_model.layers)
print(f"Número de layers do ResNet50: {total_layers}")

In [ ]:
pretrained_model.trainable = True
for layer in pretrained_model.layers[:110]:
    layer.trainable = False

In [ ]:
optimizer = optimizers.RMSprop(learning_rate = optimizers.schedules.CosineDecay(0.000001, 500))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
arquivo_modelo = "modelo.h5"
checkpointer = ModelCheckpoint(arquivo_modelo,
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True)

history_final = model.fit(train_prefetch,
                          validation_data=(val_prefetch),
                          epochs = 10,
                          callbacks=[checkpointer],
                          initial_epoch=history.epoch[-1])
plota_historico_modelo(history_final)

# Testando modelo

In [ ]:
image_batch, label_batch = test_prefetch.as_numpy_iterator().next()
predicted_labels = np.argmax(model.predict(image_batch), axis=1)

In [ ]:
label_vs_prediction = np.transpose(np.vstack((label_batch, predicted_labels)))
print(label_vs_prediction)

In [ ]:
for image, true_label, predicted_label in zip(image_batch, label_batch, predicted_labels):
    if true_label != predicted_label:
        true_class_name = class_names[true_label]
        predicted_class_name = class_names[predicted_label]

        plt.figure()
        plt.imshow(image.astype("uint8"))
        plt.title(f'True Label: {true_class_name}, Predicted Label: {predicted_class_name}')
        plt.axis('off')

plt.show()

# Matriz de confusão

In [ ]:
model = tf.keras.models.load_model('modelo.h5')

def get_true_and_predicted_labels(model, dataset):
    true_labels = []
    predicted_labels = []

    for images, labels in dataset:
        true_labels.extend(tf.argmax(labels, axis=1))
        predicted_labels.extend(np.argmax(model.predict(images), axis=1))

    return true_labels, predicted_labels

true_labels, predicted_labels = get_true_and_predicted_labels(model, val_prefetch)

confusion = confusion_matrix(true_labels, predicted_labels)

In [ ]:
def plot_confusion_matrix(confusion, class_names):
    plt.imshow(confusion, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()

    tick_marks = np.arange(len(class_names))
    
    plt.xticks(tick_marks, class_names, rotation=45, ha='right') 
    plt.yticks(tick_marks, class_names)
    
    fmt = 'd'
    thresh = confusion.max() / 2.
    
    for i, j in itertools.product(range(confusion.shape[0]), range(confusion.shape[1])):
        plt.text(j, i, format(confusion[i, j], fmt),
                horizontalalignment="center",
                color="white" if confusion[i, j] > thresh else "black")
    
    plt.ylabel('Correct Classification')
    plt.xlabel('Prediction')

plt.figure(figsize=(12, 10)) 

plot_confusion_matrix(confusion, class_names)
plt.show()